<a href="https://colab.research.google.com/github/denisshaf/DSPractice/blob/DS-2.1/notebooks/Data_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Practice DS - 2.1] Baseline modeling pt.1

In [ ]:
import os, gc

import pandas as pd
import numpy as np
import dask.dataframe as dd

from matplotlib import pyplot as plt
import seaborn as sns

## MyDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ln -s '/content/drive/MyDrive' '/mydrive'

## Data/target leakage check, validation schema

- [ ] `avg_price` - average monthly price of a particular item in a particular shop; agg by month -> consider month as a group; might be predicted as *time series*

- [ ] `item_cnt_month` - **target** - agg by month -> consider month as a group

- [ ] `item_revenue` - revenue from a particular item in a particular shop = `avg_price` * `item_cnt_month`; might be predicted as *time series*. If it's predicted as time series and is known at the moment of target prediction, it won't be a *target leakage*.

- [ ] `shop_month_revenue` - monthly revenue in a particular shop from all items = groupby(`shop_id`).sum(`item_revenue`); might be predicted as *time series*. If it's predicted as time series and is known at the moment of target prediction, it won't be a *target leakage*.

- [ ] `relative_delta_revenue` = (`item_revenue` - `avg_item_revenue`) / `avg_item_revenue` (`avg_item_revenue` - averaged by all months revenue from a particular item in a particular shop (groupby `shop_id`, `item_id`)). If `item_revenue` isn't a *target leakage*, then it won't be either

- [ ] `relative_shop_delta_revenue` = (`shop_month_revenue` - `avg_shop_month_revenue`) / `avg_shop_month_revenue` (`avg_shop_month_revenue` - averaged by all months monthly revenue in a particular shop (groupby `shop_id`, `item_id`)). If `shop_month_revenue` isn't a *target leakage*, then it won't be either





In [ ]:
dtypes = {'name2': 'object', 'name3': 'object'}
prt = dd.read_csv('/mydrive/DSPractice/data/eda/features.csv/00.part', dtype=dtypes)
df = prt.compute()

In [ ]:
df

,date_block_num,shop_id,item_id,avg_price,item_cnt_month,item_revenue,shop_month_revenue,relative_delta_revenue,relative_shop_delta_revenue,shop_month_revenue_lag_1,...,relative_shop_delta_revenue_lag_12,month,year,city,type,item_category_id,name2,name3,category,subcategory
0,0.0,0,32,221.0,6.0,1326.0,2966101.0,0.9766,-0.1580,NaN,...,NaN,0,0,Якутск,Магазин,40,0,0,Кино,DVD
1,0.0,1,32,221.0,7.0,1547.0,1527054.2,4.1200,-0.2798,NaN,...,NaN,0,0,Якутск,ТЦ,40,0,0,Кино,DVD
2,0.0,2,32,0.0,0.0,0.0,1086190.2,-1.0000,-0.1632,NaN,...,NaN,0,0,Адыгея,ТЦ,40,0,0,Кино,DVD
3,0.0,3,32,349.0,3.0,1047.0,560612.0,8.8000,-0.3677,NaN,...,NaN,0,0,Балашиха,ТРК,40,0,0,Кино,DVD
4,0.0,4,32,349.0,2.0,698.0,1434996.5,3.0720,0.2018,NaN,...,NaN,0,0,Волжский,ТЦ,40,0,0,Кино,DVD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479091,0.0,33,20273,0.0,0.0,0.0,0.0,0.0000,-1.0000,NaN,...,NaN,0,0,Мытищи,ТРК,53,0,0,Книги,Художественная литература
479092,0.0,34,20273,0.0,0.0,0.0,0.0,0.0000,-1.0000,NaN,...,NaN,0,0,Новгород,ТРЦ,53,0,0,Книги,Художественная литература
479093,0.0,36,20273,0.0,0.0,0.0,0.0,0.0000,-1.0000,NaN,...,NaN,0,0,Новосибирск,ТРЦ,53,0,0,Книги,Художественная литература
479094,0.0,39,20273,0.0,0.0,0.0,0.0,0.0000,-1.0000,NaN,...,NaN,0,0,РостовНаДону,ТРК,53,0,0,Книги,Художественная литература


`avg_price`, `item_revenue` and `shop_month_revenue` might be predicted using time series.

As `avg_price` and `item_revenue` grouped by [`shop_id`, `item_id`], we have to train count(`shop_id`) * count(`item_id`) = about 80000 ts-models, so it's too expensive. Averaging items' prices by shop leads to the fact that we have to train a model for each item (20000), it's too many too.

Meanwhile we heed to train only 40 model (not few either) for `shop_month_revenue`. So maybe we'll predict only this feature useing time series.

As a baseline we can use the last value or mean value instead of time-series model.